### purpose

train RDA on first 225 seeds for the nuisance experiments

first create some directories and files with python then use R to estimate offset

### notes

I use a combination of updating MVP_12 to train nuisance RDA and MVP_19 to validate nuisance RDA.

In [1]:
from pythonimports import *

import MVP_01_train_gradient_forests as mvp01
import MVP_summary_functions as mvp

t0 = dt.now()  # notebook timer

run = '0-225'  # batch ID

nuis_outerdirs = {
    'ISO-PSsd' : f'/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_{run}',
    'ISO-TSsd-PSsd' : f'/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_{run}'
}

rda_dirs = {}
for nuis_envs, nuis_outerdir in nuis_outerdirs.items():
    rda_dirs[nuis_envs] = makedir(f'{nuis_outerdir}/rda')
    makedir(f'{nuis_outerdir}/rda/rda_files')
    
old_outerdir = f'/home/b.lind/offsets/run_20220919_{run}'
cmd_dir = f'{old_outerdir}/rda/rda_catfiles'  # where other commands are to edit
run = op.basename(old_outerdir)

mvp.latest_commit()
session_info.show()

#########################################################
Today:	June 20, 2023 - 15:14:07 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit eb8d204ff9a40e48b07c184bcc4ff8621804cde6  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Jun 15 14:34:48 2023 -0400

Current commit of MVP_offsets:
commit 024290cb8c961841537c81afd1a02917b24bc9af  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Mon Jun 12 10:46:56 2023 -0400
#########################################################



# simlink Rout_muts_full.txt

for MVP_12_rda_offset.R

In [2]:
for nuis_envs, nuis_outerdir in nuis_outerdirs.items():
    run = op.basename(nuis_outerdir.split('_')[-1])
    print(run)
    original_slimdir = f'/home/b.lind/offsets/run_20220919_{run}/slimdir'
    
    assert op.exists(original_slimdir)
    
    files = fs(original_slimdir, pattern='Rout_muts_full.txt')
    assert len(files) == 225
    
    for src in files:
        basename = op.basename(src)
        dst = f'{nuis_outerdir}/slimdir/{basename}'
        if not op.exists(dst):
            os.symlink(src, dst)

0-225
0-225


# create commands

Use previously created RDA commands and update script and args, save to new files.

In [3]:
# get original commands
rda_cmd_files = fs(cmd_dir, endswith='rda_commands.txt')

assert len(rda_cmd_files) == 225

In [4]:
cmd_files = defaultdict(list)
all_cmds = []
for nuis_envs, nuis_outerdir in nuis_outerdirs.items():
    cat_dir = makedir(f'{nuis_outerdir}/rda/rda_catfiles')
    
    num_nuis_envs = len(nuis_envs.split('-'))
    total_envs = num_nuis_envs + 2  # X nuis envs + sal + temp

    cmd_counter = Counter()
    for f in pbar(rda_cmd_files, desc=nuis_envs):
        cmds = read(f, lines=True)

        new_cmds = []
        for cmd in cmds:
            new_cmd = cmd.split()
            assert len(new_cmd) == 8
            
            # skip indidivual stuff
            if 'pool' not in new_cmd[5]:
                continue

            # change the R script used
            assert new_cmd[1] == 'MVP_12_RDA_offset.R'
            new_cmd[1] = 'MVP_nuisance_RDA_offset.R'
            
            # change the slimdir (it is not used by MVP_climate_outlier_RDA_offset.R)
            assert new_cmd[3] == f'{old_outerdir}/slimdir'
            new_cmd[3] = f'{nuis_outerdir}/slimdir'

            # change the outer directory for saving and lookups (remember there are symlinks in outlier_outerdir to outerdir)
            assert op.realpath(new_cmd[4]) == op.realpath(old_outerdir), f'{new_cmd[4] = } {old_outerdir = }'
            new_cmd[4] = nuis_outerdir
            
            # change the path to the RDS file (to be created)
            assert new_cmd[5].startswith(f'{old_outerdir}/') and new_cmd[5].endswith('.RDS')
            new_cmd[5] = new_cmd[5].replace(old_outerdir, nuis_outerdir)
            
            # update ntraits to the total number here
                # passed to MVP_12.R::standardize_envdata
                # standardize_envdata uses this to limit the number of envs in envdata
                # `ntraits` is also passed to train_outlier_rda which uses it to decide how many envs to train with
            assert int(new_cmd[7]) == float(new_cmd[7])
            ntraits = total_envs  # nuis_envs + (sal + temp)
            new_cmd[7] = str(ntraits)

            new_cmd.append('/home/b.lind/code/MVP-offsets/01_src')  # needed for MVP_nuisance_RDA_offset.R
            
            new_cmd.append(str(total_envs))  # total_traits passed to main

            # determine if cmd has been duplicated
            full_cmd = ' '.join(new_cmd)
            cmd_counter[full_cmd] += 1
            if cmd_counter[full_cmd] > 1:
                # some commands were originally for eg 1-trait 1-env, 1-trait 2-envs. So there will be duplicated
                # commands since both 1-trait 1-env and 1-trait 2-envs will be overwritten with the same `ntraits`
                # input argument. Just keep one of these duplicate commands
                print('found one')
                continue
            
            new_cmds.append(full_cmd)
            assert full_cmd not in all_cmds
            all_cmds.append(full_cmd)

        outlier_f = op.join(cat_dir, op.basename(f))

        assert outlier_f != f

        cmd_files[nuis_envs].append(outlier_f)

        with open(outlier_f, 'w') as o:
            o.write('\n'.join(new_cmds))


ISO-TSsd-PSsd: 100%|███████████████| 225/225 [00:01<00:00, 166.83it/s]


In [5]:
len(all_cmds), luni(all_cmds)

(3600, 3600)

In [6]:
read(cmd_files['ISO-TSsd-PSsd'][0])[0].split()

['Rscript',
 'MVP_nuisance_RDA_offset.R',
 '1231094',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/rda_files/1231094_pooled_RDA.RDS',
 'TRUE',
 '5',
 '/home/b.lind/code/MVP-offsets/01_src',
 '5']

In [7]:
read(cmd_files['ISO-PSsd'][0])[0].split()

['Rscript',
 'MVP_nuisance_RDA_offset.R',
 '1231094',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/slimdir',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/rda_files/1231094_pooled_RDA.RDS',
 'TRUE',
 '4',
 '/home/b.lind/code/MVP-offsets/01_src',
 '4']

# create shfiles

In [8]:
for nuis_envs, nuis_outerdir in nuis_outerdirs.items():
    print(f'cd {nuis_outerdir}/rda')

cd /home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda
cd /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda


In [9]:
shfiles = defaultdict(list)
for nuis_envs, files in cmd_files.items():
    shdir = makedir(f'{rda_dirs[nuis_envs]}/shfiles')
    
    outerdir = nuis_outerdirs[nuis_envs]
    
    nuis_slimdir = f'{outerdir}/slimdir'
    assert op.exists(nuis_slimdir)
    
    for cmd_file in pbar(files, desc=nuis_envs):
        seed = op.basename(cmd_file).split("_")[0]
        job = f'{nuis_envs}_{run}_' + op.basename(cmd_file).rstrip('.txt')
        text = f'''#!/bin/bash
#SBATCH --job-name={job}
#SBATCH --time=1:00:00
#SBATCH --mem=9000
#SBATCH --partition=lotterhos
#SBATCH --nodes=1
#SBATCH --cpus-per-task=16
#SBATCH --output={job}_%j.out
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL

source $HOME/.bashrc

module load parallel

conda activate MVP_env_R4.0.3

cd /home/b.lind/code/MVP-offsets/01_src

Rscript /home/b.lind/code/MVP-offsets/01_src/MVP_pooled_pca_and_rda.R \\
{seed} \\
{nuis_slimdir} \\
{outerdir}/gradient_forests/training/training_files/{seed}_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all.txt \\
{outerdir} \\
/home/b.lind/code/MVP-offsets/01_src # /full/path.R is necessary

# run RDA offset estimation
cat {cmd_file} | parallel -j 16 --progress --eta

conda activate mvp_env
python MVP_nuisance_rda_validation.py {seed} {nuis_slimdir} {outerdir}

'''

        shfile = op.join(shdir, f'{job}.sh')

        with open(shfile, 'w') as o:
            o.write(text)

        shfiles[nuis_envs].append(shfile)

    assert len(shfiles[nuis_envs]) == 225

ISO-TSsd-PSsd: 100%|███████████████| 225/225 [00:00<00:00, 284.65it/s]


In [10]:
shfiles[nuis_envs][0]

'/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/shfiles/ISO-TSsd-PSsd_0-225_1231094_rda_commands.sh'

In [11]:
print(
    read(shfiles[nuis_envs][0],
         lines=False)
)

#!/bin/bash
#SBATCH --job-name=ISO-TSsd-PSsd_0-225_1231094_rda_commands
#SBATCH --time=1:00:00
#SBATCH --mem=9000
#SBATCH --partition=lotterhos
#SBATCH --nodes=1
#SBATCH --cpus-per-task=16
#SBATCH --output=ISO-TSsd-PSsd_0-225_1231094_rda_commands_%j.out
#SBATCH --mail-user=b.lind@northeastern.edu
#SBATCH --mail-type=FAIL

source $HOME/.bashrc

module load parallel

conda activate MVP_env_R4.0.3

cd /home/b.lind/code/MVP-offsets/01_src

Rscript /home/b.lind/code/MVP-offsets/01_src/MVP_pooled_pca_and_rda.R \
1231094 \
/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir \
/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/gradient_forests/training/training_files/1231094_Rout_Gmat_sample_maf-gt-p01_GFready_pooled_all.txt \
/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225 \
/home/b.lind/code/MVP-offsets/01_src # /full/path.R is necessary

# run RDA offset estimation
cat /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/rda_catfil

In [12]:
lines = read('/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/rda_catfiles/1231094_rda_commands.txt')

lines[0].split()

['Rscript',
 'MVP_nuisance_RDA_offset.R',
 '1231094',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/rda_files/1231094_pooled_RDA.RDS',
 'TRUE',
 '5',
 '/home/b.lind/code/MVP-offsets/01_src',
 '5']

# sbatch

In [13]:
# sbatch the files
pids = {}
for nuis_envs, files in shfiles.items():
    print(nuis_envs)
    
    pids[nuis_envs] = sbatch(files)

ISO-PSsd


sbatching: 100%|███████████████| 225/225 [01:03<00:00,  3.55it/s]


ISO-TSsd-PSsd


sbatching: 100%|███████████████| 225/225 [01:09<00:00,  3.23it/s]


In [14]:
# # get an alert when they're all done
# for nuis_envs, jobs in pids.items():
#     outerdir = nuis_outerdirs[nuis_envs]
    
# #     # i can set outerdir to whatever I want - it's not used in MVP_19
# #         # it's only used to check basename between outerdir and outlier_outerdir input args 
# #     outerdir = '/dummy/path/%s' % op.basename(outlier_outerdir)

    
    
#     create_watcherfile(
#         jobs,
#         directory=shdir,
#         watcher_name=f'{nuis_envs}_rda_outlier_watcher',
#         end_alert=True,
#         time='2:00:00',
#         ntasks=1,
#         mem='4000',
#         rem_flags=['#SBATCH --nodes=1', '#SBATCH --cpus-per-task=36'],
#         added_text = '\n'.join(['',
#                                 'source $HOME/.bashrc',
#                                 '',
#                                 'conda activate mvp_env',
#                                 '',
#                                 'cd $HOME/code/MVP-offsets/01_src',
#                                 ''
#                                 f'python MVP_nuisance_rda_validation.py {outerdir} {outlier_outerdir}'
#                                 ''
#                                ])
#     )

In [15]:
formatclock(dt.now() - t0)

'0-00:02:18'

In [16]:
Squeue()

🗒️  Queue Summary:

{'lotterhos': {'PD': 446, 'R': 1}}

In [17]:
Squeue(partition='lotterhos', states='PD').update(to_partition='long', num_jobs=0.4)

update: 100%|███████████████| 179/179 [00:20<00:00,  8.86it/s]


In [22]:
Squeue(partition='lotterhos', states='PD').update(to_partition='short', num_jobs=180)

update: 100%|███████████████| 180/180 [00:18<00:00,  9.99it/s]


In [5]:
class sq(Squeue):
    def watch(self, sleep=1, progress_bar=True):
        """Refresh __repr__ every `sleep` seconds, clear previous printout."""
        if progress_bar is True:
            sleeper = pyimp.sleeping
        else:
            sleeper = time.sleep
        
        try:
            while True:
                print(self)
                sleeper(sleep)
                clear_output(wait=True)
                
        except KeyboardInterrupt as e:
            pass
        
        pass

In [20]:
Squeue().watch(sleep=5)

🗒️  Queue Summary:

{'long': {'PD': 105, 'R': 5},
 'lotterhos': {'PD': 57, 'R': 3},
 'short': {'PD': 117, 'R': 6}}


sleeping:  20%|███            | 1/5 [00:01<00:07,  1.91s/it]

KeyboardInterrupt after 1 seconds of sleep.


In [22]:
Squeue(grepping='rda', states='PD').update(timelimit='0:10:00')

update: 100%|███████████████| 279/279 [00:30<00:00,  9.19it/s]


In [24]:
Squeue()

🗒️  Queue Summary:

{'long': {'R': 2},
 'lotterhos': {'PD': 13, 'R': 3},
 'short': {'PD': 9, 'R': 7}}

In [3]:
Squeue().watch()

🗒️  Queue Summary:

{'lotterhos': {'R': 1}}


sleeping: 100%|███████████████| 5/5 [00:05<00:00,  1.00s/it]


# see which commands had errors

I have a feeling the ones that had errors were those with some low number of loci that went into training the rda, which then mucked up the RDA, as I found out in some of the shuffled and shuffled nuisance runs

A: yes they all had low loci numbers, I will skip these when validating

### note

some how I ended up adding `ntraits-4` keys to the `ISO-TSsd-PSsd` (ie `ntraits-5`) dict, which added erroroneous scores to the perforamnce dict and thus to the `{outerdirs['ISO-TSsd-PSsd']}/summaries/all_peformance_dicts/level_scores.pkl`

in the next section below, I go back in and remove these erroneous keys, resave the pkl and then re-execute 02_analysis/07_experiments/02_nuisance_envs/07_save_level_scores_replicates/01_save_level_scores_nuisance_ISO-TSsd-PSsd.ipynb

In [18]:
# get the commands that did not produce outfiles - inspired by mvp19.check_completed
needed_cmds = []
for nuis_envs, nuis_outerdir in nuis_outerdirs.items():
    all_files = []
    
    print(nuis_envs)
    cat_dir = f'{nuis_outerdir}/rda/rda_catfiles'
    shdir = f'{nuis_outerdir}/rda/shfiles'
    offset_dir = f'{nuis_outerdir}/rda/offset_outfiles'
    
    catfiles = fs(cat_dir, endswith='_rda_commands.txt')
    shfiles = fs(shdir, endswith='.sh', exclude='watcher')
    
    assert len(catfiles) == len(shfiles), (len(catfiles) , len(shfiles))
        
    for cat in catfiles:
        cmds = read(cat, lines=True)

        for cmd in cmds:
            if 'pooled' not in cmd:  # ignore individual data
                continue

            # THIS PART IS DIFFERENT THAN mvp19.check_completed
            rscript, script, seed, *args, rds, use_RDA_outliers, ntraits, mvp_dir, total_traits = cmd.split()

            structcorr = 'structcorr' if 'structcorr' in rds else 'nocorr'

            basename = f'{seed}_pooled_{use_RDA_outliers}_ntraits-{ntraits}_{structcorr}_rda_offset.txt'
            f = op.join(offset_dir, basename)

            assert basename not in all_files, basename
            all_files.append(basename)

            if not op.exists(f):
                needed_cmds.append(cmd)

            if ntraits == '1':
                f = f.replace('ntraits-1', 'ntraits-2')
                assert op.basename(f) not in all_files
                all_files.append(op.basename(f))

                if not op.exists(f):
                    needed_cmds.append(cmd)
                    
    print(len(needed_cmds))

ISO-PSsd
4
ISO-TSsd-PSsd
19


In [19]:
# which marker sets were behind these errors
marker_sets = Counter()
for cmd in needed_cmds:
    marker_set = cmd.split()[-4]
    marker_sets[marker_set] += 1
    
marker_sets

Counter({'CAUSAL': 15, 'TRUE': 4})

In [20]:
# determine the number of outlier markers - inspired by MVP_12_RDA_offset.R::subset_snps
for cmd in needed_cmds:
    marker_set = cmd.split()[-4]
    outerdir = cmd.split()[-6]
    seed = cmd.split()[2]
    rds = op.basename(cmd.split()[-5])
    
    if marker_set == 'CAUSAL':
        mutsfile = f'{outerdir}/slimdir/{seed}_Rout_muts_full.txt'  # symlink to original data
        muts = pd.read_table(mutsfile, delim_whitespace=True)
        
        outliers = muts['mutname'][muts['mutID'] != 1]
        
    elif marker_set == 'TRUE':
        mutsfile = f'{outerdir}/pca/mutfiles/{seed}_pooled_Rout_muts_full.txt'
        muts = pd.read_table(mutsfile, delim_whitespace=True)
        
        if 'structcorr' in rds:
            col = 'RDA_mlog10P_sig_corr'
        else:
            col = 'RDA_mlog10P_sig'
            
        outliers = muts['mutname'][muts[col]=='TRUE']
#     elif marker_set == 'NEUTRAL':
#         mutsfile = f'{outerdir}/slimdir/{seed}_Rout_muts_full.txt'  # symlink to original data
#         muts = pd.read_table(mutsfile, delim_whitespace=True)
        
#         outliers = muts['mutname'][(muts.causal_temp == 'neutral') & (muts.causal_sal == 'neutral')]
    else:
        raise Exception('crap')
        

    print(seed, marker_set, len(outliers), op.basename(outerdir))

1231254 CAUSAL 3 ISO-PSsd_0-225
1231254 CAUSAL 3 ISO-PSsd_0-225
1231275 CAUSAL 3 ISO-PSsd_0-225
1231275 CAUSAL 3 ISO-PSsd_0-225
1231188 TRUE 0 ISO-TSsd-PSsd_0-225
1231218 TRUE 0 ISO-TSsd-PSsd_0-225
1231245 CAUSAL 5 ISO-TSsd-PSsd_0-225
1231250 CAUSAL 4 ISO-TSsd-PSsd_0-225
1231250 CAUSAL 4 ISO-TSsd-PSsd_0-225
1231253 CAUSAL 4 ISO-TSsd-PSsd_0-225
1231253 CAUSAL 4 ISO-TSsd-PSsd_0-225
1231254 CAUSAL 3 ISO-TSsd-PSsd_0-225
1231254 CAUSAL 3 ISO-TSsd-PSsd_0-225
1231275 CAUSAL 3 ISO-TSsd-PSsd_0-225
1231275 CAUSAL 3 ISO-TSsd-PSsd_0-225
1231293 TRUE 0 ISO-TSsd-PSsd_0-225
1231313 TRUE 0 ISO-TSsd-PSsd_0-225
1231314 CAUSAL 4 ISO-TSsd-PSsd_0-225
1231314 CAUSAL 4 ISO-TSsd-PSsd_0-225


# create dummy files so that code can be executed

In [27]:
import MVP_03_validate_gradient_forests as mvp03

In [29]:
pd.read_table('/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231167_pooled_TRUE_ntraits-4_nocorr_rda_offset.txt',
              delim_whitespace=True)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0.000000,0.166779,0.420439,0.486583,0.644809,0.747382,0.620234,0.605247,0.625412,0.558805,0.280651,0.354539,0.503361,0.485704,0.595133,0.601092,0.493786,0.513082,0.568529,0.442987,0.465234,0.427552,0.463034,0.470988,0.568967,0.514972,0.350957,0.493254,0.498003,0.409284,0.655059,0.507660,0.570862,0.573900,0.589521,0.455057,0.425563,0.487498,0.532846,0.517922,0.861327,0.581889,0.788657,0.636272,0.629626,0.536595,0.493174,0.576955,0.618524,0.792653,1.249561,0.723093,0.908790,0.729019,0.737902,0.665540,0.676476,0.684861,0.799233,1.101264,1.405083,1.008268,0.927771,0.820039,0.836036,0.813964,0.826797,0.885837,1.239853,1.651504,1.510406,1.201070,1.197589,0.980258,0.971207,1.004143,1.086576,1.299908,1.679758,1.917841,1.505291,1.407135,1.347230,1.172726,1.142847,1.184939,1.344802,1.779350,2.061869,2.142469,1.538641,1.583975,1.562004,1.363854,1.375797,1.491355,1.854104,2.188092,2.250791,2.231466
2,0.166779,0.000000,0.254300,0.319851,0.485825,0.606467,0.497658,0.475456,0.476791,0.403798,0.215698,0.203737,0.343889,0.322754,0.431320,0.452404,0.370991,0.360167,0.404942,0.294163,0.379412,0.301072,0.328016,0.327103,0.413726,0.371784,0.237260,0.329029,0.353725,0.322991,0.603097,0.418859,0.495245,0.444714,0.447270,0.331098,0.290799,0.350609,0.438495,0.490949,0.880616,0.547452,0.758836,0.520191,0.509266,0.439050,0.391528,0.479885,0.592054,0.843039,1.286210,0.720774,0.881267,0.642393,0.652259,0.579715,0.600716,0.622481,0.811290,1.168296,1.441697,1.009573,0.885698,0.754246,0.761181,0.740980,0.769951,0.872883,1.303418,1.738961,1.540077,1.197362,1.174991,0.929962,0.914101,0.956013,1.091084,1.346005,1.755547,2.005474,1.522815,1.399971,1.321890,1.133037,1.100598,1.153007,1.369031,1.845808,2.142936,2.229847,1.549457,1.573146,1.546220,1.336479,1.347890,1.498990,1.909178,2.262395,2.333584,2.317639
3,0.420439,0.254300,0.000000,0.075683,0.269701,0.434502,0.387447,0.346323,0.282983,0.184020,0.319607,0.139602,0.138854,0.118841,0.206902,0.284435,0.300813,0.188762,0.159060,0.145300,0.371475,0.240774,0.234829,0.213458,0.232414,0.251447,0.268874,0.115051,0.210562,0.336740,0.608103,0.399869,0.484606,0.334378,0.305074,0.287162,0.236399,0.246973,0.402386,0.554942,0.968395,0.594953,0.787440,0.427353,0.413030,0.412575,0.368769,0.434401,0.644329,0.974977,1.381909,0.794472,0.904343,0.591982,0.605055,0.544625,0.577529,0.616764,0.897193,1.309642,1.534019,1.068036,0.886653,0.729646,0.721928,0.707503,0.759101,0.920806,1.438473,1.897594,1.620782,1.240273,1.190922,0.917279,0.893027,0.946756,1.154185,1.457734,1.899318,2.161991,1.586837,1.431093,1.328423,1.126323,1.093004,1.159255,1.450748,1.976607,2.290109,2.384519,1.603626,1.594420,1.561501,1.341553,1.352893,1.553942,2.024159,2.399942,2.482050,2.470443
4,0.486583,0.319851,0.075683,0.000000,0.210131,0.389542,0.366613,0.328409,0.258326,0.185221,0.379429,0.193989,0.129826,0.087631,0.133367,0.235866,0.290557,0.163013,0.120005,0.186728,0.407950,0.270117,0.247416,0.209221,0.186096,0.215898,0.283600,0.067029,0.220197,0.379074,0.634957,0.422526,0.502197,0.322669,0.271859,0.276667,0.231081,0.246281,0.421515,0.592865,1.003682,0.622399,0.807845,0.415559,0.393008,0.408756,0.371325,0.440170,0.672338,1.016872,1.415938,0.822801,0.921323,0.588542,0.598186,0.538710,0.579721,0.627126,0.927601,1.350795,1.566025,1.091981,0.896850,0.730177,0.715618,0.702133,0.762851,0.939962,1.475627,1.940278,1.649508,1.259960,1.203379,0.919750,0.890633,0.947586,1.173070,1.488226,1.937449,2.203577,1.611232,1.447476,1.337902,1.129833,1.093740,1.163521,1.473084,2.010584,2.328454,2.425151,1.624918,1.607956,1.572247,1.347355,1.356950,1.569105,2.053459,2.435082,2.520033,2.510024
5,0.644809,0.485825,0.269701,0.210131,0.000000,0.194158,0.243329,0.212995,0.161430,0.272030,0.58535

In [74]:
# save needed files as a blank dataframe with 0's - this will result in np.nan correlations since all values are 0
df = pd.DataFrame(index=range(1,101,1), columns=range(1,101,1))
df.index = df.index.astype(str)
df.columns = df.columns.astype(str)
for col in df.columns:
    df[col] = 0

files_created = []
for cmd in needed_cmds:
    marker_set = cmd.split()[-4]
    outerdir = cmd.split()[-6]
    seed = cmd.split()[2]
    rds = op.basename(cmd.split()[-5])
    nuis_envs = op.basename(outerdir).split("_")[0]
    
    ntraits = 2 + len(nuis_envs.split('-'))
    
    offset_dir = f'/home/b.lind/offsets/run_20220919_nuisance/{nuis_envs}_0-225/rda/offset_outfiles'
    f1 = f'{offset_dir}/{seed}_pooled_{marker_set}_ntraits-{ntraits}_structcorr_rda_offset.txt'
    f2 = f1.replace('_structcorr_', '_nocorr_')
    
    assert f1 != f2
    
    for f in [f1, f2]:
        df.to_csv(f, sep=' ', index=True, header=True)
        files_created.append(f)
        
len(files_created), luni(files_created)

(38, 24)

In [59]:
# run validation commands
pycmds = []
for cmd in needed_cmds:
    marker_set = cmd.split()[-4]
    outerdir = cmd.split()[-6]
    seed = cmd.split()[2]
    rds = op.basename(cmd.split()[-5])
    nuis_envs = op.basename(outerdir).split("_")[0]
    
    pycmds.append(f'python MVP_nuisance_rda_validation.py {seed} {outerdir}/slimdir {outerdir}\n')
    
os.chdir('/home/b.lind/code/MVP-offsets/01_src')
    
for cmd in uni(pycmds):
    print(cmd)
    os.system(cmd)

python MVP_nuisance_rda_validation.py 1231254 /home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/slimdir /home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225



/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)



Retrieving offset data ...
1231254 pooled CAUSAL ntraits-4 nocorr
1231254 pooled CAUSAL ntraits-4 structcorr
1231254 pooled FALSE ntraits-4 nocorr
1231254 pooled FALSE ntraits-4 structcorr
1231254 pooled NEUTRAL ntraits-4 nocorr
1231254 pooled NEUTRAL ntraits-4 structcorr
1231254 pooled TRUE ntraits-4 nocorr
1231254 pooled TRUE ntraits-4 structcorr

	wrote offset_dfs to : /home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/validation/offset_dfs/1231254_offset_dfs.pkl

Reading in info for subsampled individuals ...

Retrieving fitness data ...

Calculating performance ...
	wrote performance_dicts to : /home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/validation/corrs/1231254_performance_dicts.pkl

DONE!!
	time to complete: 0-00:00:02

python MVP_nuisance_rda_validation.py 1231275 /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225



/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)



Retrieving offset data ...
1231275 pooled CAUSAL ntraits-5 nocorr
1231275 pooled CAUSAL ntraits-5 structcorr
1231275 pooled FALSE ntraits-5 nocorr
1231275 pooled FALSE ntraits-5 structcorr
1231275 pooled NEUTRAL ntraits-5 nocorr
1231275 pooled NEUTRAL ntraits-5 structcorr
1231275 pooled TRUE ntraits-5 nocorr
1231275 pooled TRUE ntraits-5 structcorr

	wrote offset_dfs to : /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/validation/offset_dfs/1231275_offset_dfs.pkl

Reading in info for subsampled individuals ...

Retrieving fitness data ...

Calculating performance ...
	wrote performance_dicts to : /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/validation/corrs/1231275_performance_dicts.pkl

DONE!!
	time to complete: 0-00:00:02

python MVP_nuisance_rda_validation.py 1231245 /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225


Retrieving offset data ...
1231245 pooled

...

/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)


python MVP_nuisance_rda_validation.py 1231250 /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225



/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)



Retrieving offset data ...
1231250 pooled CAUSAL ntraits-5 nocorr
1231250 pooled CAUSAL ntraits-5 structcorr
1231250 pooled FALSE ntraits-5 nocorr
1231250 pooled FALSE ntraits-5 structcorr
1231250 pooled NEUTRAL ntraits-5 nocorr
1231250 pooled NEUTRAL ntraits-5 structcorr
1231250 pooled TRUE ntraits-5 nocorr
1231250 pooled TRUE ntraits-5 structcorr

	wrote offset_dfs to : /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/validation/offset_dfs/1231250_offset_dfs.pkl

Reading in info for subsampled individuals ...

Retrieving fitness data ...

Calculating performance ...
	wrote performance_dicts to : /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/validation/corrs/1231250_performance_dicts.pkl

DONE!!
	time to complete: 0-00:00:02

python MVP_nuisance_rda_validation.py 1231218 /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir /home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225


Retrieving offset data ...
1231218 pooled

/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning: invalid value encountered in double_scalars
  slope = ssxym / ssxm
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  slope_stderr = np.sqrt((1 - r**2) * ssym / ssxm / df)
/home/b.lind/anaconda3/envs/mvp_env/lib/python3.8/site-packages/scipy/stats/_stats_mstats_common.py:194: RuntimeWarning: invalid value encountered in double_scalars
  intercept_stderr = slope_stderr * np.sqrt(ssxm + xmean**2)


In [85]:
# sometimes there were warnings, if missing a key overwrite and save
mycopy = pklload('/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/validation/corrs/1231254_performance_dicts.pkl')['garden_performance']['pooled']['CAUSAL']['ntraits-4']['nocorr'].copy()

for cmd in pbar(needed_cmds):
    marker_set = cmd.split()[-4]
    outerdir = cmd.split()[-6]
    seed = cmd.split()[2]
    rds = op.basename(cmd.split()[-5])
    nuis_envs = op.basename(outerdir).split("_")[0]
    
    pkl = f'{outerdir}/rda/validation/corrs/{seed}_performance_dicts.pkl'
    
    corrs = pklload(pkl)
    
    for structcorr in ['nocorr', 'structcorr']:
        try:
            assert corrs['garden_performance']['pooled'][marker_set]['ntraits-4'][structcorr].isnull().all()
        except KeyError as e:
            print(f'key error {seed} {marker_set} {structcorr}')
            corrs['garden_performance']['pooled'][marker_set]['ntraits-4'][structcorr] = mycopy
            pkldump(corrs, pkl)


 89%|█████████████▍ | 17/19 [00:00<00:00, 75.61it/s]

key error 1231188 TRUE nocorr
key error 1231188 TRUE structcorr
key error 1231218 TRUE nocorr
key error 1231218 TRUE structcorr
key error 1231245 CAUSAL nocorr
key error 1231245 CAUSAL structcorr
key error 1231250 CAUSAL nocorr
key error 1231250 CAUSAL structcorr
key error 1231253 CAUSAL nocorr
key error 1231253 CAUSAL structcorr
key error 1231254 CAUSAL nocorr
key error 1231254 CAUSAL structcorr
key error 1231275 CAUSAL nocorr
key error 1231275 CAUSAL structcorr
key error 1231293 TRUE nocorr
key error 1231293 TRUE structcorr
key error 1231313 TRUE nocorr
key error 1231313 TRUE structcorr
key error 1231314 CAUSAL nocorr


100%|███████████████| 19/19 [00:00<00:00, 75.42it/s]

key error 1231314 CAUSAL structcorr


In [87]:
mycopy

1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
       ..
96    NaN
97    NaN
98    NaN
99    NaN
100   NaN
Length: 100, dtype: float64

In [86]:
# prove to myself it resulted in all np.nan values for validation scores
for cmd in pbar(needed_cmds):
    marker_set = cmd.split()[-4]
    outerdir = cmd.split()[-6]
    seed = cmd.split()[2]
    rds = op.basename(cmd.split()[-5])
    nuis_envs = op.basename(outerdir).split("_")[0]
    
    pkl = f'{outerdir}/rda/validation/corrs/{seed}_performance_dicts.pkl'
    
    corrs = pklload(pkl)
    
    for structcorr in ['nocorr', 'structcorr']:
        assert corrs['garden_performance']['pooled'][marker_set]['ntraits-4'][structcorr].isnull().all()



100%|███████████████| 19/19 [00:00<00:00, 164.88it/s]


In [41]:
files_created

# ['/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231188_pooled_TRUE_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231218_pooled_TRUE_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231245_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231250_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231250_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231253_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231253_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231293_pooled_TRUE_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231313_pooled_TRUE_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231314_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/1231314_pooled_CAUSAL_ntraits-5_structcorr_rda_offset.txt',
#  '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt']

['/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_nocorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231275_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
 '/home/b.lind/offsets/run_20220

In [1]:
from pythonimports import * 

In [3]:
pd.read_table('/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/1231254_pooled_CAUSAL_ntraits-4_structcorr_rda_offset.txt',
             delim_whitespace=True)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
d = '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/offset_outfiles/'

seed_files = defaultdict(list)
for f in fs(d, endswith='offset.txt'):
    seed = op.basename(f).split("_")[0]
    seed_files[seed].append(f)
    
for seed, files in seed_files.items():
    print(seed, len(files))

1231094 8
1231095 8
1231096 8
1231097 8
1231098 8
1231099 8
1231100 8
1231101 8
1231102 8
1231103 8
1231104 8
1231105 8
1231106 8
1231107 8
1231108 8
1231109 8
1231110 8
1231111 8
1231112 8
1231113 8
1231114 8
1231115 8
1231116 8
1231117 8
1231118 8
1231119 8
1231120 8
1231121 8
1231122 8
1231123 8
1231124 8
1231125 8
1231126 8
1231127 8
1231128 8
1231129 8
1231130 8
1231131 8
1231132 8
1231133 8
1231134 8
1231135 8
1231136 8
1231137 8
1231138 8
1231139 8
1231140 8
1231141 8
1231142 8
1231143 8
1231144 8
1231145 8
1231146 8
1231147 8
1231148 8
1231149 8
1231150 8
1231151 8
1231152 8
1231153 8
1231154 8
1231155 8
1231156 8
1231157 8
1231158 8
1231159 8
1231160 8
1231161 8
1231162 8
1231163 8
1231164 8
1231165 8
1231166 8
1231167 8
1231168 8
1231169 8
1231170 8
1231171 8
1231172 8
1231173 8
1231174 8
1231175 8
1231176 8
1231177 8
1231178 8
1231179 8
1231180 8
1231181 8
1231182 8
1231183 8
1231184 8
1231185 8
1231186 8
1231187 8
1231188 8
1231189 8
1231190 8
1231191 8
1231192 8
1231193 8


In [20]:
d = '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/offset_outfiles/'

seed_files = defaultdict(list)
for f in fs(d, endswith='offset.txt'):
    seed = op.basename(f).split("_")[0]
    seed_files[seed].append(f)
    
for seed, files in seed_files.items():
    print(seed, len(files))

1231094 8
1231095 8
1231096 8
1231097 8
1231098 8
1231099 8
1231100 8
1231101 8
1231102 8
1231103 8
1231104 8
1231105 8
1231106 8
1231107 8
1231108 8
1231109 8
1231110 8
1231111 8
1231112 8
1231113 8
1231114 8
1231115 8
1231116 8
1231117 8
1231118 8
1231119 8
1231120 8
1231121 8
1231122 8
1231123 8
1231124 8
1231125 8
1231126 8
1231127 8
1231128 8
1231129 8
1231130 8
1231131 8
1231132 8
1231133 8
1231134 8
1231135 8
1231136 8
1231137 8
1231138 8
1231139 8
1231140 8
1231141 8
1231142 8
1231143 8
1231144 8
1231145 8
1231146 8
1231147 8
1231148 8
1231149 8
1231150 8
1231151 8
1231152 8
1231153 8
1231154 8
1231155 8
1231156 8
1231157 8
1231158 8
1231159 8
1231160 8
1231161 8
1231162 8
1231163 8
1231164 8
1231165 8
1231166 8
1231167 8
1231168 8
1231169 8
1231170 8
1231171 8
1231172 8
1231173 8
1231174 8
1231175 8
1231176 8
1231177 8
1231178 8
1231179 8
1231180 8
1231181 8
1231182 8
1231183 8
1231184 8
1231185 8
1231186 8
1231187 8
1231188 8
1231189 8
1231190 8
1231191 8
1231192 8
1231193 8


In [21]:
# ISO-TSsd-PSsd
all_c = Counter()
for seed, files in seed_files.items():
    c = Counter()
    for f in files:
        for x in op.basename(f).split("_"):
            c[x] += 1
            all_c[x] += 1
            
all_c

Counter({'1231094': 8,
         'pooled': 1800,
         'CAUSAL': 450,
         'ntraits-5': 1800,
         'nocorr': 900,
         'rda': 1800,
         'offset.txt': 1800,
         'structcorr': 900,
         'FALSE': 450,
         'NEUTRAL': 450,
         'TRUE': 450,
         '1231095': 8,
         '1231096': 8,
         '1231097': 8,
         '1231098': 8,
         '1231099': 8,
         '1231100': 8,
         '1231101': 8,
         '1231102': 8,
         '1231103': 8,
         '1231104': 8,
         '1231105': 8,
         '1231106': 8,
         '1231107': 8,
         '1231108': 8,
         '1231109': 8,
         '1231110': 8,
         '1231111': 8,
         '1231112': 8,
         '1231113': 8,
         '1231114': 8,
         '1231115': 8,
         '1231116': 8,
         '1231117': 8,
         '1231118': 8,
         '1231119': 8,
         '1231120': 8,
         '1231121': 8,
         '1231122': 8,
         '1231123': 8,
         '1231124': 8,
         '1231125': 8,
         '1231

In [10]:
# ISO-PSsd
all_c = Counter()
for seed, files in seed_files.items():
    c = Counter()
    for f in files:
        for x in op.basename(f).split("_"):
            c[x] += 1
            all_c[x] += 1
            
all_c

Counter({'1231094': 8,
         'pooled': 1800,
         'CAUSAL': 450,
         'ntraits-4': 1800,
         'nocorr': 900,
         'rda': 1800,
         'offset.txt': 1800,
         'structcorr': 900,
         'FALSE': 450,
         'NEUTRAL': 450,
         'TRUE': 450,
         '1231095': 8,
         '1231096': 8,
         '1231097': 8,
         '1231098': 8,
         '1231099': 8,
         '1231100': 8,
         '1231101': 8,
         '1231102': 8,
         '1231103': 8,
         '1231104': 8,
         '1231105': 8,
         '1231106': 8,
         '1231107': 8,
         '1231108': 8,
         '1231109': 8,
         '1231110': 8,
         '1231111': 8,
         '1231112': 8,
         '1231113': 8,
         '1231114': 8,
         '1231115': 8,
         '1231116': 8,
         '1231117': 8,
         '1231118': 8,
         '1231119': 8,
         '1231120': 8,
         '1231121': 8,
         '1231122': 8,
         '1231123': 8,
         '1231124': 8,
         '1231125': 8,
         '1231

In [19]:
d = '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225/rda/validation/corrs'
pkls = fs(d, endswith='.pkl')

c = Counter()
for pkl in pkls:
    x = pklload(pkl)
    for args, df in unwrap_dictionary(x):
        assert isinstance(df, pd.core.series.Series)
        
        for arg in args:
            c[arg] += 1

    
c

Counter({'garden_performance': 1800,
         'pooled': 7200,
         'CAUSAL': 1800,
         'ntraits-4': 7200,
         'nocorr': 3600,
         'structcorr': 3600,
         'FALSE': 1800,
         'NEUTRAL': 1800,
         'TRUE': 1800,
         'source_performance': 1800,
         'garden_slopes': 1800,
         'source_slopes': 1800})

In [18]:
d = '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/rda/validation/corrs'
pkls = fs(d, endswith='.pkl')

c = Counter()
for pkl in pkls:
    x = pklload(pkl)
    for args, df in unwrap_dictionary(x):
        assert isinstance(df, pd.core.series.Series)
        
        for arg in args:
            c[arg] += 1

    
c

Counter({'garden_performance': 1820,
         'pooled': 7220,
         'CAUSAL': 1812,
         'ntraits-5': 7200,
         'nocorr': 3610,
         'structcorr': 3610,
         'FALSE': 1800,
         'NEUTRAL': 1800,
         'TRUE': 1808,
         'source_performance': 1800,
         'garden_slopes': 1800,
         'source_slopes': 1800,
         'ntraits-4': 20})

In [16]:
args

('source_slopes', 'pooled', 'TRUE', 'ntraits-4', 'structcorr')

In [22]:
import MVP_06_validate_RONA as mvp06

In [23]:
slimdir = '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225/slimdir'
seed = '1231318'

subset, locations, envdata = mvp06.get_pop_data(slimdir, seed)


Reading in info for subsampled individuals ...


In [24]:
subset

,sal_opt,temp_opt,ISO,TSsd,PSsd,y,x,subpopID,indID,sample_name
1,-1.0,-1.0,0.009381,-2.245441,1.003559,1.0,1.0,1,1,1
2,-0.5,-1.0,0.239071,-1.967663,0.518946,1.0,2.0,2,2,2
3,0.0,-1.0,0.580610,-0.963343,-0.246231,1.0,3.0,3,3,3
4,0.5,-1.0,0.639031,-0.164371,-0.399267,1.0,4.0,4,4,4
5,1.0,-1.0,-0.036985,-0.291426,-0.661614,1.0,5.0,5,5,5
...,...,...,...,...,...,...,...,...,...,...
96,1.0,1.0,0.113241,-0.239327,-0.054937,10.0,6.0,96,96,96
97,0.5,1.0,0.881061,0.598108,1.480884,10.0,7.0,97,97,97
98,0.0,1.0,1.587251,1.669095,2.461321,10.0,8.0,98,98,98
99,-0.5,1.0,1.381814,2.462649,2.880066,10.0,9.0,99,99,99


# try and figure out why ISO-TSsd-PSsd is collecting weirdly

some how I ended up adding ntraits-4 keys to the ISO-TSsd-PSsd (ie ntraits-5) dict, which added erroroneous scores to the perforamnce dict and thus to the `{outerdirs['ISO-TSsd-PSsd']}/summaries/all_peformance_dicts/level_scores.pkl`

in this section, I go back in and remove these erroneous keys, resave the pkl and then re-execute `02_analysis/07_experiments/02_nuisance_envs/07_save_level_scores_replicates/01_save_level_scores_nuisance_ISO-TSsd-PSsd.ipynb`

In [1]:
from pythonimports import *

lview, dview = get_client(cluster_id='1687524639-ape7', profile='lotterhos')

latest_commit()
session_info.show()

36 36
##################################################################
Today:	June 23, 2023 - 11:26:47 EDT
python version: 3.8.5
conda env: mvp_env

Current commit of pythonimports:
commit eb8d204ff9a40e48b07c184bcc4ff8621804cde6  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Jun 15 14:34:48 2023 -0400
##################################################################



In [2]:
# directories
outerdirs = {
    'ISO-PSsd' : '/home/b.lind/offsets/run_20220919_nuisance/ISO-PSsd_0-225',
    'ISO-TSsd-PSsd' : '/home/b.lind/offsets/run_20220919_nuisance/ISO-TSsd-PSsd_0-225'
}

# get unique combinations of keys in each nested dictionary

In [3]:
# read in all saved performance pkls
pkls = defaultdict(dict)
for nuis_envs, outerdir in outerdirs.items():
    files = fs(f'{outerdir}/rda/validation/corrs')
    for f in pbar(files, desc=nuis_envs):
        seed = op.basename(f).split("_")[0]
        pkls[nuis_envs][seed] = pklload(f)

ISO-TSsd-PSsd: 100%|███████████████| 225/225 [00:00<00:00, 266.48it/s]


In [4]:
# get a list of args to see what is in each pkl
combos = defaultdict(list)
for (nuis_envs, *args), x in unwrap_dictionary(pkls):
    combos[nuis_envs].append('X'.join(args))

In [5]:
len(combos['ISO-PSsd']), luni(combos['ISO-PSsd'])

(7200, 7200)

In [6]:
len(combos['ISO-TSsd-PSsd']), luni(combos['ISO-TSsd-PSsd'])

(7220, 7220)

In [7]:
# are there any combos from ntraits-5 in the ntraits-4 dicts? A: NO!
fake4 = [x.replace('ntraits-5', 'ntraits-4') for x in combos['ISO-TSsd-PSsd']]
set(combos['ISO-PSsd']) - set(fake4)

set()

In [8]:
# are there any combos from ntraits-4 in the ntraits-5 dicts? A: YES!

# these values should not be in the ntraits-5 dict
fake5 = [x.replace('ntraits-4', 'ntraits-5') for x in combos['ISO-PSsd']]
extra = set(combos['ISO-TSsd-PSsd']) - set(fake5)

extra

{'1231188Xgarden_performanceXpooledXTRUEXntraits-4Xnocorr',
 '1231188Xgarden_performanceXpooledXTRUEXntraits-4Xstructcorr',
 '1231218Xgarden_performanceXpooledXTRUEXntraits-4Xnocorr',
 '1231218Xgarden_performanceXpooledXTRUEXntraits-4Xstructcorr',
 '1231245Xgarden_performanceXpooledXCAUSALXntraits-4Xnocorr',
 '1231245Xgarden_performanceXpooledXCAUSALXntraits-4Xstructcorr',
 '1231250Xgarden_performanceXpooledXCAUSALXntraits-4Xnocorr',
 '1231250Xgarden_performanceXpooledXCAUSALXntraits-4Xstructcorr',
 '1231253Xgarden_performanceXpooledXCAUSALXntraits-4Xnocorr',
 '1231253Xgarden_performanceXpooledXCAUSALXntraits-4Xstructcorr',
 '1231254Xgarden_performanceXpooledXCAUSALXntraits-4Xnocorr',
 '1231254Xgarden_performanceXpooledXCAUSALXntraits-4Xstructcorr',
 '1231275Xgarden_performanceXpooledXCAUSALXntraits-4Xnocorr',
 '1231275Xgarden_performanceXpooledXCAUSALXntraits-4Xstructcorr',
 '1231293Xgarden_performanceXpooledXTRUEXntraits-4Xnocorr',
 '1231293Xgarden_performanceXpooledXTRUEXntraits-4Xs

In [9]:
# group args by seed to load pkls easier below
combined = defaultdict(list)
for combo in extra:
    seed, *args = combo.split('X')    
    combined[seed].append('X'.join(args))

In [10]:
combined.keys()

dict_keys(['1231218', '1231313', '1231275', '1231293', '1231250', '1231245', '1231253', '1231188', '1231314', '1231254'])

In [32]:
# # make a copy just in case
# for seed, combo_list in combined.items():
#     pkl = f'{outerdirs["ISO-TSsd-PSsd"]}/rda/validation/corrs/{seed}_performance_dicts.pkl'
#     dst = f'/home/b.lind/delete/{seed}_performance_dicts.pkl'
#     shutil.copy(pkl, dst)

In [34]:
# remove unnecessary keys, re-save
for seed, combo_list in combined.items():
    pkl = f'{outerdirs["ISO-TSsd-PSsd"]}/rda/validation/corrs/{seed}_performance_dicts.pkl'
    
    pkldict = pklload(pkl)
    
    not_needed = []
    for combo in combo_list:
        performance, pooled, marker_set, ntraits, structcorr = combo.split('X')
                
        name = 'X'.join(combo.split("X")[:-1])
        
        if name in not_needed:
            continue
        
        assert performance in keys(pkldict)
        assert pooled in keys(pkldict[performance])
        assert marker_set in keys(pkldict[performance][pooled])
        assert ntraits in keys(pkldict[performance][pooled][marker_set])
        assert ntraits == 'ntraits-4'
        
        pkldict[performance][pooled][marker_set].pop(ntraits)
        
        not_needed.append(name)
        
    pkldump(pkldict, pkl)


### redo code to find extra combos (make sure len(extra) == 0)

In [35]:
# read in all saved performance pkls
pkls = defaultdict(dict)
for nuis_envs, outerdir in outerdirs.items():
    files = fs(f'{outerdir}/rda/validation/corrs')
    for f in pbar(files, desc=nuis_envs):
        seed = op.basename(f).split("_")[0]
        pkls[nuis_envs][seed] = pklload(f)

ISO-TSsd-PSsd: 100%|███████████████| 225/225 [00:00<00:00, 253.61it/s]


In [36]:
# get a list of args to see what is in each pkl
combos = defaultdict(list)
for (nuis_envs, *args), x in unwrap_dictionary(pkls):
    combos[nuis_envs].append('X'.join(args))

In [37]:
len(combos['ISO-PSsd']), luni(combos['ISO-PSsd'])

(7200, 7200)

In [38]:
len(combos['ISO-TSsd-PSsd']), luni(combos['ISO-TSsd-PSsd'])  # 7200, good sign!!

(7200, 7200)

In [39]:
fake4 = [x.replace('ntraits-5', 'ntraits-4') for x in combos['ISO-TSsd-PSsd']]
set(combos['ISO-PSsd']) - set(fake4)

set()

In [40]:
# these values should not be in the ntraits-5 dict - YAY NOTHING!
fake5 = [x.replace('ntraits-4', 'ntraits-5') for x in combos['ISO-PSsd']]
extra = set(combos['ISO-TSsd-PSsd']) - set(fake5)

extra

set()